# Programming Assignment 1
CSCI-4930/5930 ML Spring 2019
Author: Shreya Dubey 

Importing all the libraries 

In [1]:
import numpy as np
import imageio
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
sns.set()
import sklearn
from sklearn.model_selection import train_test_split
import warnings
import math
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=DeprecationWarning)

Loading the .mat file

In [2]:
from scipy.io import loadmat
y = loadmat('wiki_labeled_1.mat')
A = y['wiki_labeled']

Reading the data from the loaded mat file.

In [3]:
full_path = A['full_path'][0][0]
AGE = A['age'][0][0]
face_score = A['face_score'][0][0]
second_face_score = A['second_face_score'][0][0]
number = full_path.size
final_full_path = full_path[0]

In [4]:
AGE_new = np.transpose(AGE)
AGE_new.shape

(60327, 1)

In [5]:
number

60327

Loading Images from the full_path from the .mat  file.

In [6]:
import cv2
import math

X = np.empty((number, 100*100))

n = 0
n1 = 0 
for i in final_full_path: 
    im_path = i[0]
    path = 'wiki_labeled/wiki_labeled/' + str(im_path)
  #checking for outliers  
    if(math.isnan(second_face_score[:,n1][0] ) and (math.isfinite(face_score[:,n1][0]))):
        im = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        X[n,:] = ( im.reshape(1,10000)  )
        n = n + 1 
        n1 = n1+1
    else:
        n1 = n1+1


KeyboardInterrupt: 

In [ ]:
X.shape

Taking only the images that are valid

In [ ]:
print('number of images = ',n)
X = X[0:n,:]

In [ ]:
X.shape

Calculating mean to centralise the data

In [ ]:
X_mean = (np.sum(X, axis = 0))/n

In [ ]:
for i in range (n):
    X[i,:] = X[i,:]-X_mean
    

Taking out the outliers from the age data

In [ ]:
AGE_new_mod = np.empty((n, 1))
num = 0
num1 = 0 
for i in range (number): 
    if(math.isnan(second_face_score[:,num1][0] ) and (math.isfinite(face_score[:,num1][0]))):
        AGE_new_mod[num,:]  = AGE_new[i,:]     
        num = num + 1 
        num1 = num1+1
    else:
        num1 = num1+1

In [ ]:
AGE_new_mod.shape


Splitting the data into 80% training and 20% test set.

In [ ]:
X_total = np.append(X,AGE_new_mod,axis=1)

In [ ]:
X_train, X_test = train_test_split(X_total, test_size=0.20)

In [ ]:
y_train = X_train[:,-1]
y_test = X_test[:,-1]
y_train.shape
y_test.shape
X_train = X_train[:,0:10000]
X_test = X_test[:,0:10000]
X_train.shape
X_test.shape

Creating the covariance matrix

In [ ]:
X_train_T = np.transpose(X_train)

In [ ]:
X_train.shape

In [ ]:
C_train = X_train_T@X_train
C_train = C_train/(n-1)

In [ ]:
C_train.shape

Calculating the eigen values and eigen vectors

In [ ]:
from numpy import linalg as LA

w,v = LA.eig(C_train)
print('v =',v)
print('w =',w)

Displaying the top 20 eigen faces

In [ ]:
fig, axs = plt.subplots(4,5, figsize=(10, 10))
fig.subplots_adjust(hspace = .5, wspace=.001)

axs = axs.ravel()

for i in range(20):
    img1 = v[:,i]
    img1 = img1.reshape(100,100)
    axs[i].imshow(img1,cmap='gray')
    

Creating the scree plot

In [ ]:
K = np.linspace(1,10000,10000)
plt.figure(figsize = (12,10))
plt.plot(K,w)
plt.xlim(-10,70)
plt.xlabel('Principal Component')
plt.ylabel('Eigen Values')
plt.show()

Creating plot between number of K values and variance

In [ ]:
Si = sum(w)
print('Total variance = ',Si)
Sk = sum(w[0:15])
print('% variance = ',(Sk/Si)*100)

SumK = np.empty((1,10000))

for i in range (10000):
    SumK[:,i] = (sum(w[0:i])/Si)*100
    
    
plt.figure(figsize = (12,10))
plt.plot(K,SumK[0])
plt.xlim(0,6000)
plt.ylim(70,100)
plt.xlabel('Principal Components')
plt.ylabel('% Variance')
plt.show()

Selecting Value of K (depending on the scree plot)

In [ ]:
K_value = 15

Projecting training data images on to the eigenfaces 

In [ ]:
X_new = v[:,0:K_value]

X_data_pca = X_train@X_new

Projecting test data images on to the eigenfaces

In [ ]:
X_test_pca = X_test@X_new

#deleting X to release memory
del X

In [ ]:
X_data_pca.shape

Normalizing data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

SC = MinMaxScaler()

X_data_pca = SC.fit_transform(X_data_pca)

X_test_pca = SC.transform(X_test_pca)

Performing Stochastic gradient descent

In [ ]:
#converts pandas Dataframe to numpy.ndarray
ones = np.ones([X_data_pca.shape[0],1])
X_data_pca = np.concatenate((ones,X_data_pca),axis=1)

y = y_train

#select initial weights. Taking all zeros to start with
weight = np.zeros([1,X_data_pca.shape[1]])

In [ ]:
#set learning rate and number of iterations
alpha = 0.00000001
iters = 12000

y_train.shape = (31122,1)

# Print shapes of X, y and theta to make sure their dimensions are correct 
print(X_data_pca.shape) ; 
print(y_train.shape) ;
print(weight.shape);

#computeLost
def computeLoss(X,y,weight):
    
    h = X@weight.T    # '@' for matrix multiplication. Alternatively matmult() can be used
    error = h-y    
    loss = np.power(error,2)
    J = np.sum(loss)/(2*len(X))
    return J

print(computeLoss(X_data_pca,y_train,weight))

m = X_data_pca.shape[0]

#gradient descent
def gradientDescent(X,y,weight,iters,alpha):
    loss = np.zeros(iters)
    for i in range(iters):
        for j in range(m):
            X_i = X[j,:]
            X_i.shape = (1,K_value+1)
            y_i = y[j,:]
            y_i.shape = (1,1)
            y_hat = X_i@weight.T
            gradient = X_i.T@(y_hat - y_i)   
            weight = weight - (alpha*gradient).T 
        loss[i] = computeLoss(X, y, weight) 
        
        # print cost after every 100 iterations to keep a track of when the cost function converges
        if i%100 == 0:
            print("Iteration %d | loss: %f" % (i, loss[i]))
            
    return weight,loss

#running the gd and cost function
W,loss = gradientDescent(X_data_pca,y_train,weight,iters,alpha)
print("Weigths for the linear model", W)

finalCost = computeLoss(X_data_pca,y_train,W)
print(finalCost)

Calculating the RMSE on test data

In [ ]:
# make the first column all ones
ones = np.ones([X_test_pca.shape[0],1])
X_test_pca = np.concatenate((ones,X_test_pca),axis=1)

In [ ]:
print("shape of X_test_pca: ", X_test_pca.shape)
print("shape of W: ", W.shape)
print("shape of y_test: ", y_test.shape)

y_test.shape = (7781,1)
y_learn = X_test_pca@W.T
error = y_test - y_learn

In [ ]:
def getRMSE(y_test,x_test,W):
    err = y_test - (x_test@W.T)  
    sq_err = err*err ; 
    mean_sqr_err = np.sum(sq_err, axis = 0 )/len(err)
    RMSE = math.sqrt(mean_sqr_err)
    return RMSE

rmse1 = getRMSE(y_test,X_test_pca,W)
print("Root Mean Squared Error = ", rmse1)

#RMSE1 = 18.062686675869838
#RMSE2 = 16.73084347718248
#RMSE3 = 16.617141336336186
#RMSE4 = 16.807692550390442
#RMSE5 = 18.352997171428015

In [ ]:
R =[18.062686675869838,16.73084347718248,16.617141336336186,16.807692550390442,18.352997171428015]
Avg_rmse = np.mean(R)
stdv_rmse = np.std(R)

print('Average RMSE = ',Avg_rmse)
print('Standard deviation of RMSE =',stdv_rmse)

In [ ]:
K_v = [2,10,20,40,50,60,80,100,200]
RMSE_v = [17.076894074696263,16.686346916598435,17.24229549812708,17.1501307646619,17.087651807277023,17.076223134162554,17.154376556864765,17.119457525152008,17.17250913355512]

plt.plot(K_v,RMSE_v,'b-')
plt.ylim(10,20)
plt.ylabel('RMSE values')
plt.xlabel('value of K')


Loading the wiki_judge data

In [ ]:
y_judge = loadmat('wiki_judgeX.mat')
A_judge = y_judge['wiki_judgeX']

In [ ]:
image_full_path = A_judge['full_path'][0][0]
image_number = image_full_path.size
final_full_path_judge = image_full_path[0]

In [ ]:
X_judge = np.empty((image_number, 100*100))

n = 0 
for i in final_full_path_judge: 
    im_path = i[0]
    path = 'wiki_judge_images/wiki_judge_images/' + str(im_path)
    im = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    X_judge[n,:] = ( im.reshape(1,10000)  )
    n = n + 1   

In [ ]:
for i in range (n):
    X_judge[i,:] = X_judge[i,:]-X_mean

In [ ]:
X_judge.shape
X_judge_pca = X_judge@X_new

In [ ]:
X_judge_pca = SC.transform(X_judge_pca)

Calculating the age values for wiki_judge data

In [ ]:
ones = np.ones([X_judge_pca.shape[0],1])
X_judge_pca = np.concatenate((ones,X_judge_pca),axis=1)
Age_output = X_judge_pca@W.T

In [ ]:
dataset = pd.DataFrame({'age':Age_output[:,0]})

dataset.to_csv('AGE_PREDICTION.csv', sep=',')